In [38]:
!pip --quiet install grpcio grpcio-tools langchain yandexcloud


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [40]:
from langchain.chains import LLMChain
from langchain_community.llms import YandexGPT
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatYandexGPT
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_core.prompts import FewShotChatMessagePromptTemplate
import re
import json
import os

import pandas as pd
import numpy as np

from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload

In [41]:
from dotenv import load_dotenv
from pathlib import Path

load_dotenv()

API_KEY = os.getenv("YANDEX_API_KEY")
MODEL_URI = os.getenv("YANDEX_MODEL_URI")

In [42]:
print(f"API_KEY set: {bool(API_KEY)}")
print(f"MODEL_URI: {MODEL_URI}")

API_KEY set: True
MODEL_URI: gpt://b1gjoiftodjh5fkultoi/yandexgpt/latest


In [43]:
gpt = YandexGPT(api_key=API_KEY, model_uri=MODEL_URI, temperature=0)

In [229]:
model = ChatYandexGPT(api_key=API_KEY, model_uri=MODEL_URI, temperature=0)

In [230]:
system_prompt = """
Ты — сельскохозяйственный анализатор данных.

Твоя задача — извлечь из текста следующую информацию и представить её строго в формате JSON:

{{
  \"дата\": str | null,
  \"операции\": [
    {{
      \"операция\": str,  # Только из утверждённого списка ниже
      \"отделение\": int | null,  # Только число или null
      \"подразделение\": str | null,  # Из утверждённого списка, может отсутствовать
      \"площадь\": {{
          \"за_день\": float | null,
          \"c_начала_операции\": float | null
      }},
      \"площадь_по_ПУ\": {{
          \"за_день\": float | null,
          \"c_начала_операции\": float | null
      }},
      \"растительная_культура\": str | null,  # Только из списка ниже
      \"вал\": {{
        \"за_день\": int | null,
        \"с_начала\": int | null
      }} 
    }}
  ]
}}

Списки для сверки:
- Операции:
    - 1-я междурядная культивация
    - 2-я междурядная культивация
    - Боронование довсходовое
    - Внесение минеральных удобрений
    - Выравнивание зяби
    - 2-е Выравнивание зяби
    - Гербицидная обработка
    - 1 Гербицидная обработка
    - 2 Гербицидная обработка
    - 3 Гербицидная обработка
    - 4 Гербицидная обработка
    - Дискование
    - 2-е Дискование
    - Инсектицидная обработка
    - Культивация
    - Пахота
    - Подкормка
    - Предпосевная культивация
    - Прикатывание посевов
    - Сев
    - Сплошная культивация
    - Уборка
    - Фунгицидная обработка
    - Чизлевание

- Подразделения:
    - АОР
    - ТСК
    - АО Кропоткинское
    - Восход
    - Колхоз Прогресс
    - Мир
    - СП Коломейцево

- Растительные культуры:
    - Вика+Тритикале
    - Горох на зерно
    - Горох товарный
    - Гуар
    - Конопля
    - Кориандр
    - Кукуруза кормовая
    - Кукуруза семенная
    - Кукуруза товарная
    - Люцерна
    - Многолетние злаковые травы
    - Многолетние травы прошлых лет
    - Многолетние травы текущего года
    - Овес
    - Подсолнечник кондитерский
    - Подсолнечник семенной
    - Подсолнечник товарный
    - Просо
    - Пшеница озимая на зеленый корм
    - Пшеница озимая семенная
    - Пшеница озимая товарная
    - Рапс озимый
    - Рапс яровой
    - Свекла сахарная
    - Сорго
    - Сорго кормовой
    - Сорго-суданковый гибрид
    - Соя семенная
    - Соя товарная
    - Чистый пар
    - Чумиза
    - Ячмень озимый
    - Ячмень озимый семенной

Правила:
- Если под строкой операции и строки "По ПУ X/Y" следуют несколько строк вида "Отд X Y/Z", они считаются частью одной и той же операции. Все отделения наследуют название операции, культуру и данные "площадь_по_ПУ".
- Если строка с «Отд X Y/Z» (отделение) расположена после строки с «По ПУ» (или её вариантов) и между ними нет новой операции, то она также относится к текущей операции. Заполни её в поле "отделение" и "площадь".
- Если в сообщении указано одно подразделение (например, \"СП Коломейцево\") в начале, до описания операций, оно применяется ко всем операциям в этом сообщении.
- Если указано только отделение числом (например, \"Отд 11\"), автоматически привязывай подразделение \"АОР\".
- Поддерживай отделения, записанные как \"Отделение X/Y\" или \"Отделение X Y/Z\" (аналогично \"Отд\").
- Если указана операция вида \"X-е диск\", \"X-е выравн\", интерпретируй их как \"X-е Дискование\" или \"X-е Выравнивание зяби\" соответственно, если это допустимо по логике.
- Если операция упоминает культуру в сокращённой форме (например, \"сах св\", \"пш\", \"подс\", \"соии\"), попытайся распознать и сопоставить с полной формой из утверждённого списка культур.
- Если в названии операции указана культура (например, \"Диск оз пшеницы\"), попытайся извлечь культуру как часть названия и соотнести её с утверждённым списком культур.
- Если указан \"Вал\" внутри описания конкретной операции (например, рядом с площадью или культурой), то он должен быть привязан именно к этой операции. В этом случае добавь поле \"вал\" внутрь соответствующего объекта операции.
- Если указано значение \"Вал\" (например, \"Вал 1259680/6660630\"), то первое число — \"за_день\", второе — \"с_начала\". Заполни соответствующее поле \"вал\".
- Если встречаются значения \"Вал за день\" и \"Вал с начала\" (например, \"Вал 58720\", \"Вал 1259680/6660630\"), извлекай их как отдельные значения, где первое число — вал за день, второе — вал с начала. Эти значения не входят в блок \"операции\", но могут быть полезны для дополнительного анализа.
- Если в сообщении указана дата (например, \"15.10\", \"14.11.2024\", \"30.03.25г\" и т.п.), извлеки её и заполни поле \"дата\". Это значение применяется ко всем операциям.
- Название операций может содержать порядковый номер (например, \"2-е дискование\", \"3-е дискование\"). Всегда преобразуй такие операции к формату \"Дискование N-е\" с правильной нумерацией, если они входят в утверждённый список или логически продолжают его.
- Если данные об отделении указаны в формате \"Отд X Y/Z\" или \"Отд X-Y/Z\", интерпретируй X как номер отделения, а Y как \"за_день\", Z как \"c_начала_операции\" для поля \"площадь\".
- Если данных недостаточно, поставь null.
- Ничего не придумывай от себя.
- Если в тексте несколько операций, каждая должна быть отдельным элементом массива.
- Поле подразделение может отсутствовать в сообщении. Если подразделение указано одно для всех операций, применяй его ко всем операциям.
- Поле отделение может быть разным для разных операций в одном сообщении.
- Если площадь указана в формате X/Y (например, 15/775 или По ПУ 15/1382), интерпретируй X как \"за_день\", а Y как \"c_начала_операции\".
- Поддерживай разные написания фразы \"По ПУ\": \"ПоПу\", \"ПУ\", \"По Пу\" — все должны трактоваться одинаково как \"площадь_по_ПУ\".
- Если в названии операции указана культура (например, «сев подсолнечника», «пахота под сою»), разбей выражение: тип операции запиши в поле "операция" (например, "Сев"), а название культуры — в "растительная_культура", если она входит в утверждённый список.
"""


In [239]:
def normalize_herbicide_operation(inputs: dict) -> dict:
    text = inputs["input"]
    text = re.sub(r"внесение\s+почв\w*\s+гербицид\w*", "внесение гербицидов", text, flags=re.IGNORECASE)
    
    return {"input": text}

In [240]:
from langchain.chains.transform import TransformChain

transform_chain_normalize = TransformChain(
    input_variables=["input"],
    output_variables=["input"],
    transform=normalize_herbicide_operation
)

In [241]:
def assign_department(inputs: dict) -> dict:
    import json
    import re

    raw_output = inputs["output"]

    if isinstance(raw_output, str):
        cleaned_output = re.sub(r"^```(?:json)?\n?", "", raw_output.strip(), flags=re.IGNORECASE)
        cleaned_output = re.sub(r"\n?```$", "", cleaned_output.strip())
        try:
            data = json.loads(cleaned_output)
        except json.JSONDecodeError:
            raise ValueError("Не удалось декодировать JSON. Получен текст:\n" + cleaned_output)
    else:
        data = raw_output

    aor_departments = {1, 3, 4, 5, 6, 7, 9, 10, 11, 12, 16, 17, 18, 19, 20}

    new_operations = []

    for op in data.get("операции", []):
        otd = op.get("отделение")
        if isinstance(otd, list):
            for idx, sub_otd in enumerate(otd):
                new_op = op.copy()
                new_op["отделение"] = sub_otd

                if isinstance(op.get("площадь", {}).get("за_день"), list):
                    new_op["площадь"] = {
                        "за_день": op["площадь"]["за_день"][idx],
                        "c_начала_операции": op["площадь"]["c_начала_операции"][idx]
                    }

                if not new_op.get("подразделение") and sub_otd in aor_departments:
                    new_op["подразделение"] = "АОР"

                new_operations.append(new_op)
        else:
            if otd in aor_departments and not op.get("подразделение"):
                op["подразделение"] = "АОР"
            new_operations.append(op)

    data["операции"] = new_operations
    return {"output": json.dumps(data, ensure_ascii=False)}

transform_chain_aor = TransformChain(
    input_variables=["output"],
    output_variables=["output"],
    transform=assign_department
)

In [243]:
from langchain.chains.sequential import SimpleSequentialChain


def init_model(message: str):
    prompt = ChatPromptTemplate.from_messages([
        SystemMessagePromptTemplate.from_template(system_prompt),
        HumanMessagePromptTemplate.from_template("{input}")
    ])
    
    llm_chain = LLMChain(prompt=prompt, llm=model)
    full_chain = SimpleSequentialChain(
        chains=[transform_chain_normalize, llm_chain, transform_chain_aor],
        input_key="input",
        output_key="output",
        verbose=True
    )
    result = full_chain.run(message)
    cleaned = re.sub(r'^```(json)?|```$', '', result, flags=re.MULTILINE).strip()

    return json.loads(cleaned)

In [244]:
message = """
Уборка свеклы 27.10.день
Отд10-124/216
По ПУ 124/3104
Вал 1259680/6660630
Урожайность 279,9/308,3
По ПУ 1259680/41630600
На завод 1811630/6430580
По ПУ 1811630/41400550
Положено в кагат 399400
Вввезено с кагата 951340
Остаток 230060
Оз-9,04/12,58
Дигестия-14,50/15,05
"""



In [245]:
out = init_model(message)
out



> Entering new SimpleSequentialChain chain...

Уборка свеклы 27.10.день
Отд10-124/216
По ПУ 124/3104
Вал 1259680/6660630
Урожайность 279,9/308,3
По ПУ 1259680/41630600
На завод 1811630/6430580
По ПУ 1811630/41400550
Положено в кагат 399400
Вввезено с кагата 951340
Остаток 230060
Оз-9,04/12,58
Дигестия-14,50/15,05

```
{
  "дата": "27.10",
  "операции": [
    {
      "операция": "Уборка",
      "отделение": 10,
      "подразделение": null,
      "площадь": {
        "за_день": 124,
        "c_начала_операции": 216
      },
      "площадь_по_ПУ": {
        "за_день": 124,
        "c_начала_операции": 3104
      },
      "растительная_культура": "Свекла сахарная",
      "вал": {
        "за_день": 1259680,
        "с_начала": 6660630
      }
    }
  ]
}
```
{"дата": "27.10", "операции": [{"операция": "Уборка", "отделение": 10, "подразделение": "АОР", "площадь": {"за_день": 124, "c_начала_операции": 216}, "площадь_по_ПУ": {"за_день": 124, "c_начала_операции": 3104}, "растительная_культур

{'дата': '27.10',
 'операции': [{'операция': 'Уборка',
   'отделение': 10,
   'подразделение': 'АОР',
   'площадь': {'за_день': 124, 'c_начала_операции': 216},
   'площадь_по_ПУ': {'за_день': 124, 'c_начала_операции': 3104},
   'растительная_культура': 'Свекла сахарная',
   'вал': {'за_день': 1259680, 'с_начала': 6660630}}]}

In [246]:
message = """
Пахота зяби под сою
По Пу 15/1382
Отд 16 15/775

Пахота зяби под мн тр
По Пу 13/527
Отд 12 13/260

2-е диск сои под пшен
По Пу 87/2076
Отд 12 87/386

2-е диск сах св под пш
По Пу 149/899
Отд 17 149/232

3-е диск подсол под пш
По Пу 47/949
Отд 11 47/47

Предп культ под оз пш
По Пу 95/1312
Отд 11 95/328"
"""

In [247]:
out = init_model(message)
out



> Entering new SimpleSequentialChain chain...

Пахота зяби под сою
По Пу 15/1382
Отд 16 15/775

Пахота зяби под мн тр
По Пу 13/527
Отд 12 13/260

2-е диск сои под пшен
По Пу 87/2076
Отд 12 87/386

2-е диск сах св под пш
По Пу 149/899
Отд 17 149/232

3-е диск подсол под пш
По Пу 47/949
Отд 11 47/47

Предп культ под оз пш
По Пу 95/1312
Отд 11 95/328"

```
{
  "дата": null,
  "операции": [
    {
      "операция": "Пахота",
      "отделение": 16,
      "подразделение": null,
      "площадь": {
        "за_день": 15,
        "c_начала_операции": 1382
      },
      "площадь_по_ПУ": {
        "за_день": 15,
        "c_начала_операции": 1382
      },
      "растительная_культура": "Соя",
      "вал": null
    },
    {
      "операция": "Пахота",
      "отделение": 12,
      "подразделение": null,
      "площадь": {
        "за_день": 13,
        "c_начала_операции": 527
      },
      "площадь_по_ПУ": {
        "за_день": 13,
        "c_начала_операции": 527
      },
      "растительная_кул

{'дата': None,
 'операции': [{'операция': 'Пахота',
   'отделение': 16,
   'подразделение': 'АОР',
   'площадь': {'за_день': 15, 'c_начала_операции': 1382},
   'площадь_по_ПУ': {'за_день': 15, 'c_начала_операции': 1382},
   'растительная_культура': 'Соя',
   'вал': None},
  {'операция': 'Пахота',
   'отделение': 12,
   'подразделение': 'АОР',
   'площадь': {'за_день': 13, 'c_начала_операции': 527},
   'площадь_по_ПУ': {'за_день': 13, 'c_начала_операции': 527},
   'растительная_культура': 'Многолетние травы',
   'вал': None},
  {'операция': 'Дискование 2-е',
   'отделение': 12,
   'подразделение': 'АОР',
   'площадь': {'за_день': 87, 'c_начала_операции': 2076},
   'площадь_по_ПУ': {'за_день': 87, 'c_начала_операции': 2076},
   'растительная_культура': ['Соя', 'Пшеница'],
   'вал': None},
  {'операция': 'Дискование 2-е',
   'отделение': 17,
   'подразделение': 'АОР',
   'площадь': {'за_день': 149, 'c_начала_операции': 899},
   'площадь_по_ПУ': {'за_день': 149, 'c_начала_операции': 899},


In [248]:
message = """
14.11 Мир
Пахота зяби под кукурузу 57 га день, 562 га от начала, 83%, 110 га остаток.
Пахота зяби под сою 60 га день, 799 га от начала, 70%, 340 га остаток.
Работало 5 агрегатов.
Выравнивание зяби под сахарную свёклу 130 га день, 874 га от начала, 92 %, 78 га остаток.
Работал 1 агрегат.
"""

In [249]:
out = init_model(message)
out



> Entering new SimpleSequentialChain chain...

14.11 Мир
Пахота зяби под кукурузу 57 га день, 562 га от начала, 83%, 110 га остаток.
Пахота зяби под сою 60 га день, 799 га от начала, 70%, 340 га остаток.
Работало 5 агрегатов.
Выравнивание зяби под сахарную свёклу 130 га день, 874 га от начала, 92 %, 78 га остаток.
Работал 1 агрегат.

```
{
  "дата": "14.11",
  "операции": [
    {
      "операция": "Пахота",
      "отделение": null,
      "подразделение": "Мир",
      "площадь": {
        "за_день": 57,
        "c_начала_операции": 562
      },
      "растительная_культура": "кукуруза",
      "вал": null
    },
    {
      "операция": "Пахота",
      "отделение": null,
      "подразделение": "Мир",
      "площадь": {
        "за_день": 60,
        "c_начала_операции": 799
      },
      "растительная_культура": "соя",
      "вал": null
    },
    {
      "операция": "Выравнивание зяби",
      "отделение": null,
      "подразделение": "Мир",
      "площадь": {
        "за_день": 130,
 

{'дата': '14.11',
 'операции': [{'операция': 'Пахота',
   'отделение': None,
   'подразделение': 'Мир',
   'площадь': {'за_день': 57, 'c_начала_операции': 562},
   'растительная_культура': 'кукуруза',
   'вал': None},
  {'операция': 'Пахота',
   'отделение': None,
   'подразделение': 'Мир',
   'площадь': {'за_день': 60, 'c_начала_операции': 799},
   'растительная_культура': 'соя',
   'вал': None},
  {'операция': 'Выравнивание зяби',
   'отделение': None,
   'подразделение': 'Мир',
   'площадь': {'за_день': 130, 'c_начала_операции': 874},
   'растительная_культура': 'сахарная свёкла',
   'вал': None}]}

In [250]:
message = """
14.11.24.
СП Коломейцево 

Пахота под сах.св.
день 29га
от начала 315га(100%)

Выравнивание под сахарную свёклу 
День 20га
От начала 125га(39%)
 
Выравнивание под подсолнечник
день 14га
от начала 14га
"""

In [251]:
out = init_model(message)
out



> Entering new SimpleSequentialChain chain...

14.11.24.
СП Коломейцево 

Пахота под сах.св.
день 29га
от начала 315га(100%)

Выравнивание под сахарную свёклу 
День 20га
От начала 125га(39%)

Выравнивание под подсолнечник
день 14га
от начала 14га

```
{
  "дата": "14.11.24",
  "операции": [
    {
      "операция": "Пахота",
      "отделение": null,
      "подразделение": "СП Коломейцево",
      "площадь": {
        "за_день": 29,
        "c_начала_операции": 315
      },
      "растительная_культура": "Сахарная свёкла"
    },
    {
      "операция": "Выравнивание зяби",
      "отделение": null,
      "подразделение": "СП Коломейцево",
      "площадь": {
        "за_день": 20,
        "c_начала_операции": 125
      },
      "растительная_культура": "Сахарная свёкла"
    },
    {
      "операция": "Выравнивание зяби",
      "отделение": null,
      "подразделение": "СП Коломейцево",
      "площадь": {
        "за_день": 14,
        "c_начала_операции": 14
      },
      "растительная_к

{'дата': '14.11.24',
 'операции': [{'операция': 'Пахота',
   'отделение': None,
   'подразделение': 'СП Коломейцево',
   'площадь': {'за_день': 29, 'c_начала_операции': 315},
   'растительная_культура': 'Сахарная свёкла'},
  {'операция': 'Выравнивание зяби',
   'отделение': None,
   'подразделение': 'СП Коломейцево',
   'площадь': {'за_день': 20, 'c_начала_операции': 125},
   'растительная_культура': 'Сахарная свёкла'},
  {'операция': 'Выравнивание зяби',
   'отделение': None,
   'подразделение': 'СП Коломейцево',
   'площадь': {'за_день': 14, 'c_начала_операции': 14},
   'растительная_культура': 'Подсолнечник'}]}

In [252]:
message = """
Пахота под сах св
По Пу 77/518
Отд 12 46/298
Отд 16 21/143
Отд 17 10/17

Чизел под оз ячмень 
По Пу 22/640
Отд 11 22/242

Чизел под оз зел корм
Отд 11 40/40

Диск оз пшеницы
По Пу 28/8872
Отд 17 28/2097

2-е диск под сах св
По Пу 189/1763
Отд 11 60/209
Отд 12 122/540
Отд 17 7/172

Диск кук силос
По Пу 6/904
Отд 11 6/229

Прик под оз ячмень
По Пу 40/498
Отд 11 40/100

Уборка сои (семенной)
Отд 11 65/65
Вал 58720
Урож 9
"""

In [253]:
out = init_model(message)
out



> Entering new SimpleSequentialChain chain...

Пахота под сах св
По Пу 77/518
Отд 12 46/298
Отд 16 21/143
Отд 17 10/17

Чизел под оз ячмень 
По Пу 22/640
Отд 11 22/242

Чизел под оз зел корм
Отд 11 40/40

Диск оз пшеницы
По Пу 28/8872
Отд 17 28/2097

2-е диск под сах св
По Пу 189/1763
Отд 11 60/209
Отд 12 122/540
Отд 17 7/172

Диск кук силос
По Пу 6/904
Отд 11 6/229

Прик под оз ячмень
По Пу 40/498
Отд 11 40/100

Уборка сои (семенной)
Отд 11 65/65
Вал 58720
Урож 9

```
{
  "дата": null,
  "операции": [
    {
      "операция": "Пахота",
      "отделение": 12,
      "подразделение": null,
      "площадь": {
        "за_день": 46,
        "c_начала_операции": 298
      },
      "площадь_по_ПУ": {
        "за_день": 77,
        "c_начала_операции": 518
      },
      "растительная_культура": "Свекла сахарная",
      "вал": null
    },
    {
      "операция": "Чизлевание",
      "отделение": null,
      "подразделение": null,
      "площадь": {
        "за_день": 22,
        "c_начала_опе

{'дата': None,
 'операции': [{'операция': 'Пахота',
   'отделение': 12,
   'подразделение': 'АОР',
   'площадь': {'за_день': 46, 'c_начала_операции': 298},
   'площадь_по_ПУ': {'за_день': 77, 'c_начала_операции': 518},
   'растительная_культура': 'Свекла сахарная',
   'вал': None},
  {'операция': 'Чизлевание',
   'отделение': None,
   'подразделение': None,
   'площадь': {'за_день': 22, 'c_начала_операции': 242},
   'площадь_по_ПУ': {'за_день': 22, 'c_начала_операции': 640},
   'растительная_культура': 'Овес яровой',
   'вал': None},
  {'операция': 'Чизлевание',
   'отделение': None,
   'подразделение': None,
   'площадь': {'за_день': 40, 'c_начала_операции': 40},
   'площадь_по_ПУ': None,
   'растительная_культура': 'Пшеница озимая на зеленый корм',
   'вал': None},
  {'операция': 'Дискование',
   'отделение': 17,
   'подразделение': 'АОР',
   'площадь': {'за_день': 28, 'c_начала_операции': 2097},
   'площадь_по_ПУ': {'за_день': 28, 'c_начала_операции': 8872},
   'растительная_культур

In [254]:
message = """
30.03.25г.
СП Коломейцево

предпосевная культивация  
  -под подсолнечник
    день 30га
    от начала 187га(91%)

сев подсолнечника 
  день+ночь 57га
  от начала 157га(77%)

Внесение почвенного гербицида по подсолнечнику 
  день 82га 
  от начала 82га (38%)
"""

In [255]:
out = init_model(message)
out



> Entering new SimpleSequentialChain chain...

30.03.25г.
СП Коломейцево

предпосевная культивация  
  -под подсолнечник
    день 30га
    от начала 187га(91%)

сев подсолнечника 
  день+ночь 57га
  от начала 157га(77%)

внесение гербицидов по подсолнечнику 
  день 82га 
  от начала 82га (38%)

```
{
  "дата": "30.03.25г",
  "операции": [
    {
      "операция": "Предпосевная культивация",
      "отделение": null,
      "подразделение": "СП Коломейцево",
      "площадь": {
        "за_день": 30,
        "c_начала_операции": 187
      },
      "растительная_культура": "Подсолнечник",
      "вал": null
    },
    {
      "операция": "Сев",
      "отделение": null,
      "подразделение": "СП Коломейцево",
      "площадь": {
        "за_день": 57,
        "c_начала_операции": 157
      },
      "растительная_культура": "Подсолнечник",
      "вал": null
    },
    {
      "операция": "Внесение гербицидов",
      "отделение": null,
      "подразделение": "СП Коломейцево",
      "площадь": 

{'дата': '30.03.25г',
 'операции': [{'операция': 'Предпосевная культивация',
   'отделение': None,
   'подразделение': 'СП Коломейцево',
   'площадь': {'за_день': 30, 'c_начала_операции': 187},
   'растительная_культура': 'Подсолнечник',
   'вал': None},
  {'операция': 'Сев',
   'отделение': None,
   'подразделение': 'СП Коломейцево',
   'площадь': {'за_день': 57, 'c_начала_операции': 157},
   'растительная_культура': 'Подсолнечник',
   'вал': None},
  {'операция': 'Внесение гербицидов',
   'отделение': None,
   'подразделение': 'СП Коломейцево',
   'площадь': {'за_день': 82, 'c_начала_операции': 82},
   'растительная_культура': 'Подсолнечник',
   'вал': None}]}

In [ ]:
df.to_excel(filename, index=False)

SCOPES = ['https://www.googleapis.com/auth/drive.file']


def authenticate():
    creds = None
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
    return creds


def upload_to_drive(filename, creds):
    service = build('drive', 'v3', credentials=creds)
    file_metadata = {'name': filename}
    media = MediaFileUpload(filename, mimetype='application/vnd.openxmlformats-officedocument.spreadsheetml.sheet')
    file = service.files().create(body=file_metadata, media_body=media, fields='id').execute()
    print(f'Файл загружен. ID: {file.get("id")}')


creds = authenticate()
upload_to_drive(filename, creds)

In [256]:
def create_dataframe(json):
    records = []
    for op in json['операции']:
        record = {
            'операция': op.get('операция'),
            'отделение': op.get('отделение') if op.get('отделение') else np.nan,
            'подразделение': op.get('подразделение') if op.get('подразделение') else np.nan,
            'растительная_культура': op.get('растительная_культура'),
            'площадь_за_день': op.get('площадь', {}).get('за_день'),
            'площадь_с_начала': op.get('площадь', {}).get('c_начала_операции'),
            'площадь_по_ПУ_за_день': op.get('площадь_по_ПУ', {}).get('за_день') if op.get('площадь_по_ПУ') else np.nan,
            'площадь_по_ПУ_с_начала': op.get('площадь_по_ПУ', {}).get('c_начала_операции') if op.get(
                'площадь_по_ПУ') else np.nan,
            'вал_за_день': op.get('вал', {}).get('за_день') if op.get('вал') else np.nan,
            'вал_с_начала': op.get('вал', {}).get('с_начала') if op.get('вал') else np.nan
        }
        records.append(record)

    return pd.DataFrame(records)



In [257]:
create_dataframe(out)

,операция,отделение,подразделение,растительная_культура,площадь_за_день,площадь_с_начала,площадь_по_ПУ_за_день,площадь_по_ПУ_с_начала,вал_за_день,вал_с_начала
0,Предпосевная культивация,NaN,СП Коломейцево,Подсолнечник,30,187,NaN,NaN,NaN,NaN
1,Сев,NaN,СП Коломейцево,Подсолнечник,57,157,NaN,NaN,NaN,NaN
2,Внесение гербицидов,NaN,СП Коломейцево,Подсолнечник,82,82,NaN,NaN,NaN,NaN
